<a href="https://colab.research.google.com/github/bd1ng/llm-xai/blob/main/embed_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #10
### Explores large language model (LLM) explainability through dimension-reduction techniques tSNE, PCA, and UMAP. Includes an evaluation of the above approaches.

### Bochu Ding

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bd1ng/llm-xai/blob/main/embed_viz.ipynb)

### Adapted from XAI tutorial by Dr. Brinnae Bent, accessible here:

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AIPI-590-XAI/Duke-AI-XAI/blob/main/explainable-ml-example-notebooks/embedding-visualization.ipynb)

# 1. Set-up

## a. Import Packages and Data

In [1]:
!pip install "numpy<2" gensim==4.3.2 matplotlib==3.7.1 scikit-learn==1.2.2 umap-learn==0.5.6 plotly==5.15.0

In [34]:
# Basic
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sentence_transformers import SentenceTransformer
import pandas as pd

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# Random words; Generated by GPT4o on March 26 at 3.19p
import random
import nltk
from nltk.corpus import brown
from nltk.probability import FreqDist

## b. Model set-up

In [9]:
# I picked potion-base-4M because it was the smallest one and all the other ones maxed out the RAM.

model = SentenceTransformer('minishlab/potion-base-4M')


In [35]:
# Passing words through the model, as it doesn't have individual word embeddings stored the way GloVe does. Selecting common random words from NLTK. Workshopped with ChatGPT 4o on March 26 @3.18pm

nltk.download('brown')
nltk.download('punkt')

brown_words = [w.lower() for w in brown.words() if w.isalpha()]
fdist = FreqDist(brown_words)

# Get most common 10,000 words
common_words = [word for word, _ in fdist.most_common(10000)]

# Filter words for length between 3 and 8 characters
filtered_words = [w for w in common_words if 3 <= len(w) <= 8]

# Randomly sample 500
random_words = random.sample(filtered_words, 500)
embeddings = model.encode(random_words)

print(f"Embeddings shape: {embeddings.shape}")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Embeddings shape: (500, 128)


# 2. Visualization

## a. PCA

In [36]:
# Apply PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

df = pd.DataFrame(embeddings_pca, columns=["PC1", "PC2"])
df["word"] = random_words

# Plot using Plotly; Edited by ChaptGpt4o on March 26 at 4:02
fig = px.scatter(
    df, x="PC1", y="PC2",
    text="word",
    width=1600,
    height=1600,
    title="PCA of Potion Embeddings",
    labels={"PC1": "Principal Component 1", "PC2": "Principal Component 2"}
)

fig.update_traces(marker=dict(size=8))
fig.show()

## b. t-distributed Stochastic Neighbor Embedding (t-SNE)

In [37]:
# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

df_tsne = pd.DataFrame(embeddings_tsne, columns=["PC1", "PC2"])
df_tsne["word"] = random_words

# Plot t-SNE results using Plotly
fig_tsne = px.scatter(
    df_tsne, x="PC1", y="PC2",
    width=1600,
    height=1600,
    text="word",
    title="t-SNE of Potion Embeddings",
    labels={'0': 'Component 1', '1': 'Component 2'}
)
fig_tsne.update_traces(marker=dict(size=8))
fig_tsne.show()

## c. Uniform Manifold Approximation and Projection (UMAP)


In [38]:
# Apply UMAP
umap_model = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
embeddings_umap = umap_model.fit_transform(embeddings)

df_umap = pd.DataFrame(embeddings_umap, columns=["PC1", "PC2"])
df_umap["word"] = random_words

# Plot UMAP results using Plotly
fig_umap = px.scatter(
    df_umap, x="PC1", y="PC2",
    width=1600,
    height=1600,
    text="word",
    title="UMAP of Potion Embeddings",
    labels={'0': 'Component 1', '1': 'Component 2'}
)
fig_umap.update_traces(marker=dict(size=8))
fig_umap.show()

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1945: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



# 3. Discussion

These three approaches (PCA, t-SNE, UMAP) have different theoretical fundamentals and should be selected with discretion based on the purpose of the exercise.


PCA focuses on capturing global linear relationships, meaning that it best reflects how points relate to the entire dataset, instead of local neighbors. That's why PCA is notably less dispersed than t-SNE and UMAP in the visualizations above.  

t-SNE, on the other hand, focuses on preserving local relationships. This means that it is far better at identifying groups and clusters of similar datapoints. However, it does not preserve the distance between these clusters well.  For example, in the above visualization, "high," "top," "tall" are clustered together next to the cluster of "versions" and "original." The relationship between points within these clusters are clear, but the closeness of the two clusters likely does not tell us about their relationship as groups.

Finally UMAP utilizes non-linear dimension reduction to create clusters, which allows it to capture more complex relationships. It also attempts to preserve global structures, which allows for a better understanding of how clusters relate to each other than t-SNE. It can be employed to illuminate more complex (non-linear) relationships and balance local and global fidelity.